# 通过目标检测模型对水果图片进行检测并计价



# 一、项目成果展示

# **用于检测的图片**
![](https://ai-studio-static-online.cdn.bcebos.com/a32b0e102b464667b7739458e53cac9acb70a2ef5c4a4a4981f964dd7252dc79)

# **图片预测出的水果**
![](https://ai-studio-static-online.cdn.bcebos.com/7b1478f87c124934b45595f8fe594a91f5f25d289d984866bacd86135941871b)

# **图中水果的计价**
![](https://ai-studio-static-online.cdn.bcebos.com/7e7f76491ffd43a3b44672a4bb8113191b8025202c9e478bb15cf4185e83b183)





# 二、项目介绍


## 1.1 项目简介：
***  **这个项目产生是由于对水果自动计价的兴趣，可以方便管理者管理也能方便用户购买水果*****

## 2.2 项目亮点：
a、通过编写公式和调用方法对数据集读取与划分，且把数据和图片及标注文件移动到data文件夹下便于后续操作



b、通过使用paddlex模型库调用模型对水果图片进行检测，达到方便快捷进行目标检测的目的



c、在模型的预测方面，通过使用图片预测结果中的‘score’值对预测结果进行筛选，从而找到质量偏高的预测结果




d、通过if的嵌套语句对图片检测结果的信息筛选，且统计水果价格
**
**

In [ ]:
!pip install paddlex==2.0rc

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 450kB 13.1MB/s eta 0:00:01
     |████████████████████████████████| 307kB 21.2MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 14.5MB/s eta 0:00:01
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=278365 sha256=152b8a0cb9bd424a820f8b73d00784b4e9d0caadf6df2967969ce7da8c6892d7
  Stored in directory: /home/aistudio/.cache/pip/wheels/fb/44/67/8baa69040569b1edbd7776ec6f82c387663e724908aaa60963
Successfully built pycocotools


**首先导入项目所需要的部分库**

In [ ]:
from paddlex.det import transforms
from sklearn.utils import shuffle
import os
import pandas as pd
import numpy as np
from PIL import Image
import paddle
import paddle.nn as nn
from paddle.io import Dataset
import paddle.vision.transforms as T
import paddle.nn.functional as F
from paddle.metric import Accuracy
import random

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/distributed/parallel.py:120: UserWarning: Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything.
  "Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything."


[08-12 21:33:46 MainThread @utils.py:79] WRN paddlepaddle version: 2.1.2. The dynamic graph version of PARL is under development, not fully tested and supported


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pyarrow/pandas_compat.py:1027: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  'floating': np.float,
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of

Your running script needs PaddleX<2.0.0, please refer to https://github.com/PaddlePaddle/PaddleX/tree/release/2.0-rc/tutorials/train#%E7%89%88%E6%9C%AC%E5%8D%87%E7%BA%A7 to solve this issue.


SystemExit: -1

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3273: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# 三、数据集


# 3.1 数据集介绍
**用于这个项目的数据是从天池大数据竞赛的网站中找到的，是一个包含212张图片的VOC格式的数据集，可以方便的进行模型训练**
![](https://ai-studio-static-online.cdn.bcebos.com/e3b11c56078840f5bea042ad85bf4b294aea1e7f424b4f4db880572d2f14aa6c)

![](https://ai-studio-static-online.cdn.bcebos.com/997884f396fd4145a86e3af8c09ea65abff7c423026f421898dd5486d24505e0)




# 3.2 对数据进行处理

In [ ]:
#首先对数据解压缩
!unzip -oq /home/aistudio/work/fruitdetection_datastes.zip

In [ ]:
#对数据集进行划分，制作对应的训练集和测试集
import os
import shutil

from tqdm import tqdm
from random import shuffle

dataset = ''
train_txt = os.path.join(dataset, 'train_val.txt')
val_txt = os.path.join(dataset, 'val.txt')

#可以制作一个该文件
lbl_txt = os.path.join(dataset, 'label_list.txt')


#在dataset的基础上的文件名称
xml_base = 'annotations'
img_base = 'images'

#读取所有的xml文件
xmls = [v for v in os.listdir(os.path.join(dataset, xml_base)) if v.endswith('.xml')]
shuffle(xmls)

split = int(0.85 * len(xmls)) #划分训练集与验证集

#创建train_txt，是一个空的txt文件
with open(train_txt, 'w') as f:
    for x in tqdm(xmls[:split]):
        m = x[:-4]+'.png'
        xml_path = os.path.join(xml_base, x)
        img_path = os.path.join(img_base, m)
        f.write('{} {}\n'.format(img_path, xml_path))
    
with open(val_txt, 'w') as f:
    for x in tqdm(xmls[split:]):
        m = x[:-4]+'.png'
        xml_path = os.path.join(xml_base, x)
        img_path = os.path.join(img_base, m)
        f.write('{} {}\n'.format(img_path, xml_path))

100%|██████████| 30/30 [00:00<00:00, 144465.12it/s]


In [ ]:
#把对应的文件移动到data文件夹下
!mv images data 
!mv annotations data

In [ ]:
!mv train_val.txt data
!mv val.txt data

# 四、模型训练

# 4.1 模型的选择

2.使用PaddleX训练模型

![](https://ai-studio-static-online.cdn.bcebos.com/28e62618af89486dab83e211e8b8e9178363f005df5143f5a8bd324294676c42)

![](https://ai-studio-static-online.cdn.bcebos.com/c5bb80c39d3d4903a71a71d1fb33d1fcd780c7eab0464d6b95b11bdc7cb3d86d)


# 4.2加载预训练模型

YOLOv3模型的训练接口示例，函数内置了piecewise学习率衰减策略和momentum优化器

In [ ]:
#导入模型训练所需要的库
import paddlex as pdx
from paddlex import transforms as T

In [ ]:
# 定义训练和验证时的transforms
train_transforms = T.Compose([
    T.MixupImage(mixup_epoch=250), T.RandomDistort(),
    T.RandomExpand(im_padding_value=[123.675, 116.28, 103.53]), T.RandomCrop(),
    T.RandomHorizontalFlip(), T.BatchRandomResize(
        target_sizes=[320, 352, 384, 416, 448, 480, 512, 544, 576, 608],
        interp='RANDOM'), T.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

eval_transforms = T.Compose([
    T.Resize(
        608, interp='CUBIC'), T.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 4.3 配置数据集

In [ ]:
# 定义训练和验证所用的数据集
train_dataset = pdx.datasets.VOCDetection(
    data_dir='data',
    file_list='data/train_val.txt',
    label_list='work/labels.txt',
    transforms=train_transforms,
    shuffle=True)

eval_dataset = pdx.datasets.VOCDetection(
    data_dir='data',
    file_list='data/val.txt',
    label_list='work/labels.txt',
    transforms=eval_transforms,
    shuffle=False)

2021-08-12 21:34:54 [INFO]	Starting to read file list from dataset...
2021-08-12 21:34:55 [INFO]	170 samples in file data/train_val.txt
creating index...
index created!
2021-08-12 21:34:55 [INFO]	Starting to read file list from dataset...
2021-08-12 21:34:55 [INFO]	30 samples in file data/val.txt
creating index...
index created!


# 4.4初始化模型

In [ ]:
num_classes = len(train_dataset.labels)
model = pdx.models.YOLOv3(num_classes=num_classes, backbone='MobileNetV3_ssld')

# 4.5开始模型训练

In [12]:
#开始模型训练
model.train(
    num_epochs=270,
    train_dataset=train_dataset,
    train_batch_size=2,
    eval_dataset=eval_dataset,
    learning_rate=0.001 / 8,
    warmup_steps=1000,
    warmup_start_lr=0.0,
    save_interval_epochs=1,
    lr_decay_epochs=[216, 243],
    save_dir='output/yolov3_mobilenet')

2021-08-12 21:35:04 [INFO]	Loading pretrained model from output/yolov3_mobilenet/pretrain/MobileNetV3_large_x1_0_ssld_pretrained.pdparams
2021-08-12 21:35:05 [WARNING]	neck.yolo_block.0.conv_module.conv0.conv.weight is not in pretrained model
2021-08-12 21:35:05 [WARNING]	neck.yolo_block.0.conv_module.conv0.batch_norm.weight is not in pretrained model
2021-08-12 21:35:05 [WARNING]	neck.yolo_block.0.conv_module.conv0.batch_norm.bias is not in pretrained model
2021-08-12 21:35:05 [WARNING]	neck.yolo_block.0.conv_module.conv0.batch_norm._mean is not in pretrained model
2021-08-12 21:35:05 [WARNING]	neck.yolo_block.0.conv_module.conv0.batch_norm._variance is not in pretrained model
2021-08-12 21:35:05 [WARNING]	neck.yolo_block.0.conv_module.conv1.conv.weight is not in pretrained model
2021-08-12 21:35:05 [WARNING]	neck.yolo_block.0.conv_module.conv1.batch_norm.weight is not in pretrained model
2021-08-12 21:35:05 [WARNING]	neck.yolo_block.0.conv_module.conv1.batch_norm.bias is not in pretr

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:641: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")


2021-08-12 21:35:07 [INFO]	[TRAIN] Epoch=1/270, Step=10/85, loss_xy=13.097692, loss_wh=25.262461, loss_obj=4139.744141, loss_cls=15.364826, loss=4193.469238, lr=0.000001, time_each_step=0.17s, eta=1:3:53
2021-08-12 21:35:08 [INFO]	[TRAIN] Epoch=1/270, Step=20/85, loss_xy=9.192924, loss_wh=14.471283, loss_obj=584.513062, loss_cls=13.647904, loss=621.825134, lr=0.000002, time_each_step=0.12s, eta=0:44:58
2021-08-12 21:35:09 [INFO]	[TRAIN] Epoch=1/270, Step=30/85, loss_xy=12.207762, loss_wh=25.710129, loss_obj=124.785957, loss_cls=11.500080, loss=174.203934, lr=0.000004, time_each_step=0.11s, eta=0:42:29
2021-08-12 21:35:10 [INFO]	[TRAIN] Epoch=1/270, Step=40/85, loss_xy=9.577546, loss_wh=13.322750, loss_obj=43.656162, loss_cls=8.018109, loss=74.574570, lr=0.000005, time_each_step=0.11s, eta=0:43:18
2021-08-12 21:35:11 [INFO]	[TRAIN] Epoch=1/270, Step=50/85, loss_xy=11.298496, loss_wh=10.606833, loss_obj=46.289680, loss_cls=11.249975, loss=79.444984, lr=0.000006, time_each_step=0.11s, eta

Exception in thread Thread-35:
Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py", line 462, in _get_data
    data = self._data_queue.get(timeout=self._timeout)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/multiprocessing/queues.py", line 105, in get
    raise Empty
_queue.Empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py", line 391, in _thread_loop
    batch = self._get_data()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/

KeyboardInterrupt: 

# 五、模型预测和项目功能实现

# 5.1模型预测
**通过一段时间的模型训练，loss值达到了比较低的范围，就可以开始模型的预测了**

In [13]:
#模型预测
import glob
import numpy as np
import threading
import time
import random
import os
import base64
import cv2
import json
import paddlex as pdx
# 传入待预测图片
image_name = 'work/fruit167.png'
# 模型保存位置
model = pdx.load_model('output/yolov3_mobilenet/best_model')

img = cv2.imread(image_name)
result = model.predict(img)
cname=[]
f = open('./output/yolov3_mobilenet/result.txt', 'a')
for dt in np.array(result):
    score = dt['score']
    if score < 0.2:
        continue
    cname.append(dt['category'])

print(cname)


2021-08-12 21:46:16 [INFO]	Model[YOLOv3] loaded.
['snake fruit', 'dragon fruit', 'banana']


In [14]:

pdx.visualize_detection(
    image_name, result, threshold=0.5, save_dir='./output/yolov3_mobilenet')

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/cbook/__init__.py:2349: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  if isinstance(obj, collections.Iterator):


2021-08-12 21:47:16 [INFO]	The visualized result is saved at ./output/yolov3_mobilenet/visualize_fruit167.png


# 5.2项目功能实现

In [22]:
#项目功能实现(对水果进行计价)

pineapple_price = 'Price1'
snakefruit_price = 'Price2'
dragonfruit_price = 'Price3'
banana_price = 'Price4'

Price_num=''

for fruit in cname:
    if fruit == 'pineapple':
        Price_num=Price_num+' '+pineapple_price
    elif fruit == 'snake fruit':
        Price_num=Price_num+' '+snakefruit_price
    elif fruit == 'dragon fruit':
        Price_num=Price_num+' '+dragonfruit_price
    else:
        Price_num=Price_num+' '+banana_price

print(Price_num)



 Price2 Price3 Price4


# 六、项目总结和升华

**这个项目是在飞桨领航团、AI达人创造营的课程中完成的，包含了许多自己的想法，同时也参考了老师的课件内容。通过这个项目，自己对深度学习有了基础的认识，能够完成简单的任务。**

 **在项目最初的制作的过程中，先产生想法自己大致要做一个什么样的项目，然后在不同网站上对数据集进行寻找，在操作过程中由于数据集的格式问题更换过一个数据集。**
 
  **在项目的主要制作过程中，先是不了解文件路径，在助教的教导下，对应旁边的文件夹去运行老师的课件或者别人的项目，去逐渐熟悉文件的路径，逐渐完成了模型的运行。**
  
  **模型的预测部分和项目功能部分主要是围绕自己制作项目的点子出发，根据模型提供的结果去读取自己想要的信息。**
  
  **这个项目让自己有了对深度学习的基础理解，产生了对深度学习的兴趣，期待这个项目能让你喜欢。**